In [1]:
import rasterio
import numpy as np
import os
from rasterio.warp import calculate_default_transform, reproject, Resampling

variable = 'Velocity'
profiles = 'Idealised'
catchment = 'WykeBeck'
catchment_short = 'WB'

if catchment == 'WykeBeck':
    os.chdir(f"/nfs/a161/gy17m2a/PhD/FloodModelling/{catchment}Models/Model_{profiles}Profiles_export/Model_{profiles}Profiles_export/")
else:
    os.chdir(f"/nfs/a161/gy17m2a/PhD/FloodModelling/{catchment}Models/{catchment}Models/Model_{profiles}Profiles/")

In [210]:
# List of file paths for your 9 raster files
# filenames = [f'6h_c{i}' for i in range(1, 16)]
filenames = ['6h_sp_fl_0.1', '6h_sp_fl_0.2','6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5', '6h_sp_bl_0.6', '6h_sp_bl_0.7', '6h_sp_bl_0.8', '6h_sp_bl_0.9']
print(filenames)

if catchment == 'LinDyke':
    file_paths = [f"{filename}/{variable} (Max).Resampled.Terrain.tif" for filename in filenames] 
else:
    file_paths = [f"{filename}/{variable} (Max).Terrain.wykeDEM.tif" for filename in filenames]  # Adjust filenames if necessary

    # ## Reorder
# old_values = ['6h_c1', '6h_c8', '6h_c15', '6h_c3', '6h_c11', '6h_c10',  '6h_c9',  '6h_c13','6h_c6', '6h_c2', '6h_c12', '6h_c14', '6h_c4',  '6h_c7','6h_c5']
# new_values = ['O1',    'O2',    'O3',      'O4',    'O5',      'O6',     'O7',     'O8',    'O9',    'O10',    'O11',    'O12',   'O13',    'O14',  'O15']
# mapping = dict(zip(new_values, old_values))
# file_paths = [f"{mapping[value]}/Velocity (Max).Terrain.wykeDEM.tif" for value in new_values]

# Initialize a list to store raster data
rasters = []

# Read all rasters
for file in file_paths:
    with rasterio.open(file) as src:
        rasters.append(src.read(1))  # Read the first band (assumes single band rasters)

# Convert to numpy array (this will create a 3D array of shape (num_files, height, width))
rasters_array = np.array(rasters)

# Replace -9999 values with np.nan in the entire array
rasters_array[rasters_array == -9999] = np.nan

# Set values < 0.1 to np.nan
rasters_array[rasters_array < 0.1] = np.nan

max_indices = np.empty(rasters_array.shape[1:], dtype=float)

# Loop over each pixel position (vectorized solutions are possible with masking too)
for i in range(rasters_array.shape[1]):
    for j in range(rasters_array.shape[2]):
        # Extract the values for this pixel from all rasters
        pixel_values = rasters_array[:, i, j]
        # Check if all values are nan
        if np.all(np.isnan(pixel_values)):
            max_indices[i, j] = np.nan
        else:
            max_indices[i, j] = np.nanargmax(pixel_values) + 1  # Using np.nanargmax if needed

print(max_indices)

# Read metadata from the first raster
with rasterio.open(file_paths[0]) as src:
    profile = src.profile

# Save the result as a new GeoTIFF
output_file = f"max_depth_file_index_{catchment_short}_{profiles}_{variable}.tif"
with rasterio.open(output_file, 'w', **profile) as dst:
    dst.write(max_indices, 1)

print(f"Output file saved as {output_file}")


['6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5', '6h_sp_bl_0.6', '6h_sp_bl_0.7', '6h_sp_bl_0.8', '6h_sp_bl_0.9']
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Output file saved as max_depth_file_index_WB_Idealised_Depth.tif


In [211]:
# Define correct CRS (update this based on your actual data!)
original_crs = "EPSG:27700"  # Example: British National Grid
target_crs = "EPSG:3857"  # Convert to Web Mercator for basemap compatibility

with rasterio.open(f"max_depth_file_index_{catchment_short}_{profiles}_{variable}.tif") as src:
    transform, width, height = calculate_default_transform(
        original_crs, target_crs, src.width, src.height, *src.bounds
    )
    profile = src.profile
    profile.update({"crs": target_crs, "transform": transform, "width": width, "height": height})

    with rasterio.open(f"reprojected_max_depth_{catchment_short}_{profiles}_{variable}.tif", "w", **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=original_crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest,
            )

print(f"Reprojected raster saved as reprojected_max_depth_{catchment_short}_{profiles}_{variable}.tif")

Reprojected raster saved as reprojected_max_depth_WB_Idealised_Depth.tif


### Filtering velocity by depth

In [2]:
# List of file paths for your 9 raster files
filenames = ['6h_sp_fl_0.1', '6h_sp_fl_0.2','6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5', '6h_sp_bl_0.6', '6h_sp_bl_0.7', '6h_sp_bl_0.8', '6h_sp_bl_0.9']

print(filenames)

# Adjust based on catchment
if catchment == 'LinDyke':
    file_paths = [f"{filename}/{variable} (Max).Resampled.Terrain.tif" for filename in filenames] 
else:
    file_paths = [f"{filename}/{variable} (Max).Terrain.wykeDEM.tif" for filename in filenames]

# Initialize a list to store raster data
rasters = []

# Read all rasters
for file in file_paths:
    with rasterio.open(file) as src:
        rasters.append(src.read(1))  # Read the first band (assumes single band rasters)

# Convert to numpy array (this will create a 3D array of shape (num_files, height, width))
rasters_array = np.array(rasters)

# Replace -9999 values with np.nan in the entire array
rasters_array[rasters_array == -9999] = np.nan

# Set values < 0.1 to np.nan for all variables
# rasters_array[rasters_array < 0.1] = np.nan

# If the variable is "Velocity", set NaN in Velocity where Depth is less than 0.1
if variable == 'Velocity':
    if catchment == 'LinDyke':
        depth_file_paths = [f"{filename}/Depth (Max).Resampled.Terrain.tif" for filename in filenames] 
    else:
        depth_file_paths = [f"{filename}/Depth (Max).Terrain.wykeDEM.tif" for filename in filenames]    
    
    depth_rasters = []

    # Read Depth rasters
    for depth_file in depth_file_paths:
        with rasterio.open(depth_file) as src:
            depth_rasters.append(src.read(1))

    depth_rasters_array = np.array(depth_rasters)

    # Check where Depth is less than 0.1 and set corresponding Velocity values to NaN
    for i in range(rasters_array.shape[1]):
        for j in range(rasters_array.shape[2]):
            # Get Depth and Velocity values at the same pixel
            depth_values = depth_rasters_array[:, i, j]
            velocity_values = rasters_array[:, i, j]

            # If the Depth is less than 0.1, set the corresponding Velocity to NaN
            if np.any(depth_values < 0.1):
                rasters_array[:, i, j] = np.nan  # Set all velocity values for this pixel to NaN

# Now proceed with finding the max depth indices as before
max_indices = np.empty(rasters_array.shape[1:], dtype=float)

# Loop over each pixel position (vectorized solutions are possible with masking too)
for i in range(rasters_array.shape[1]):
    for j in range(rasters_array.shape[2]):
        # Extract the values for this pixel from all rasters
        pixel_values = rasters_array[:, i, j]
        # Check if all values are nan
        if np.all(np.isnan(pixel_values)):
            max_indices[i, j] = np.nan
        else:
            max_indices[i, j] = np.nanargmax(pixel_values) + 1  # Using np.nanargmax if needed

print(max_indices)

# Read metadata from the first raster
with rasterio.open(file_paths[0]) as src:
    profile = src.profile

# Save the result as a new GeoTIFF
output_file = f"max_depth_file_index_{catchment_short}_{profiles}_{variable}.tif"
with rasterio.open(output_file, 'w', **profile) as dst:
    dst.write(max_indices, 1)

print(f"Output file saved as {output_file}")


['6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5', '6h_sp_bl_0.6', '6h_sp_bl_0.7', '6h_sp_bl_0.8', '6h_sp_bl_0.9']
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Output file saved as max_depth_file_index_WB_Idealised_Velocity.tif
